In [1]:
]activate .

In [2]:
using DifferentialEquations
using StaticArrays
using BenchmarkTools

@inbounds @inline function ż(z, p, t)
    A, B, D = p
    p₀, p₂ = z[SVector{2}(1:2)]
    q₀, q₂ = z[SVector{2}(3:4)]

    return SVector{4}(
        -A * q₀ - 3 * B / √2 * (q₂^2 - q₀^2) - D * q₀ * (q₀^2 + q₂^2),
        -q₂ * (A + 3 * √2 * B * q₀ + D * (q₀^2 + q₂^2)),
        A * p₀,
        A * p₂
    )
end

@inbounds @inline function ṗ(p, q, params, t)
    A, B, D = params
    dp1 = -A * q[1] - 3 * B / √2 * (q[2]^2 - q[1]^2) - D * q[1] * (q[1]^2 + q[2]^2)
    dp2 = -q[2] * (A + 3 * √2 * B * q[1] + D * (q[1]^2 + q[2]^2))
    return SVector{2}(dp1, dp2)
end

@inbounds @inline function q̇(p, q, params, t)
    params.A * p
end

q̇ (generic function with 1 method)

In [3]:
q0 = SVector{2}([0.0, -4.363920590485035])
p0 = SVector{2}([10.923918825236079, -5.393598858645495])
z0 = vcat(p0, q0)
p = (A=1,B=0.55,D=0.4)

tspan = (0., 10.)

(0.0, 10.0)

In [4]:
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 10.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [5]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10);

  855.046 μs (39946 allocations: 1.55 MiB)
  369.494 μs (17753 allocations: 534.38 KiB)
  1.523 ms (110196 allocations: 3.05 MiB)


In [6]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  792.279 μs (38974 allocations: 1.21 MiB)
  354.098 μs (17439 allocations: 463.05 KiB)
  1.425 ms (108191 allocations: 2.76 MiB)


In [7]:
function step_integ(integ, t)
    while integ.t < t
        step!(integ)
    end
end

step_integ (generic function with 1 method)

In [9]:
@btime init($prob1, Vern9(), abstol=1e-14, reltol=1e-14)
@btime step_integ(integ1, $tspan[2]) setup=(integ1=init($prob1, Vern9(), abstol=1e-14, reltol=1e-14))
@btime init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime step_integ(integ2, $tspan[2]) setup=(integ2=init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14))
@btime init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10)
@btime step_integ(integ3, $tspan[2]) setup=(integ3=init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10))

  7.500 μs (86 allocations: 65.16 KiB)
  18.313 μs (866 allocations: 33.02 KiB)
  7.632 μs (95 allocations: 57.94 KiB)
  1.814 μs (88 allocations: 2.38 KiB)
  5.000 μs (79 allocations: 53.03 KiB)
  146.810 μs (11011 allocations: 306.58 KiB)


In [10]:
tspan = (0., 100.)
prob1 = ODEProblem(ż, z0, tspan, p)
prob2 = DynamicalODEProblem(ṗ, q̇, p0, q0, tspan, p)

ODEProblem with uType Tuple{SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}} and tType Float64. In-place: false
timespan: (0.0, 100.0)
u0: ([10.9239, -5.3936], [0.0, -4.36392])

In [11]:
@btime solve($prob1, Vern9(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14, save_everystep=false);
@btime solve($prob2, KahanLi8(), dt=1e-2, maxiters=1e10, save_everystep=false);

  7.979 ms (386012 allocations: 11.80 MiB)
  3.513 ms (173906 allocations: 4.43 MiB)
  16.902 ms (1080082 allocations: 27.47 MiB)


In [12]:
@btime init($prob1, Vern9(), abstol=1e-14, reltol=1e-14)
@btime step_integ(integ1, $tspan[2]) setup=(integ1=init($prob1, Vern9(), abstol=1e-14, reltol=1e-14))
@btime init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14)
@btime step_integ(integ2, $tspan[2]) setup=(integ2=init($prob2, DPRKN12(), abstol=1e-14, reltol=1e-14))
@btime init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10)
@btime step_integ(integ3, $tspan[2]) setup=(integ3=init($prob2, KahanLi8(), dt=1e-2, maxiters=1e10))

  7.501 μs (86 allocations: 65.16 KiB)
  8.545 ms (395577 allocations: 15.36 MiB)
  7.632 μs (94 allocations: 57.84 KiB)
  451.505 μs (22113 allocations: 607.76 KiB)
  9.869 μs (78 allocations: 474.81 KiB)
  18.072 ms (1100000 allocations: 28.99 MiB)
